In [2]:
import numpy as np
from keras import applications, callbacks
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers
from keras.applications import vgg16

import keras.backend as K

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls

drive  sample_data


In [5]:
datasetPath = 'drive/MyDrive/Capstone Project/dataset_binaries/'
savedModelPath = 'drive/MyDrive/Capstone Project/saved_models_binaries/'

In [6]:
x_train=np.load(datasetPath + 'x_train_np.npy')
x_cv=np.load(datasetPath + 'x_cv_np.npy')
y_train=np.load(datasetPath + 'y_train_np.npy')
y_cv=np.load(datasetPath + 'y_cv_np.npy')

VGG16 Bottleneck

In [ ]:
model = vgg16.VGG16(weights='imagenet', include_top=False)


x_train_bottleneck = model.predict(x_train)
x_cv_bottleneck = model.predict(x_cv)

np.save(datasetPath + 'x_train_bottleneck.npy', x_train_bottleneck)
np.save(datasetPath + 'x_cv_bottleneck.npy', x_cv_bottleneck)

print(x_train_bottleneck[0].shape)

58900480/58889256 [==============================] - 1s 0us/step
(2, 2, 512)


In [ ]:
x_train_bottleneck[0]

array([[[ 0.       ,  0.       ,  0.       , ...,  0.       ,
          4.7718086,  0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,
          7.3983874,  0.       ]],

       [[51.17156  ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ],
        [15.45649  ,  0.       ,  0.       , ...,  0.       ,
          0.       ,  0.       ]]], dtype=float32)

In [ ]:
x_train_bottleneck = np.load(datasetPath + 'x_train_bottleneck.npy')
x_cv_bottleneck = np.load(datasetPath + 'x_cv_bottleneck.npy')

In [ ]:
x_train_bottleneck.shape[1:]

(2, 2, 512)

Top Model

In [ ]:
top_model=Sequential()
top_model.add(Flatten(input_shape=x_train_bottleneck.shape[1:]))
top_model.add(Dense(64, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
top_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                131136    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 131,201
Trainable params: 131,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class Custom_lr(callbacks.Callback):

    def on_train_begin(self, logs={}):
	    K.set_value(self.model.optimizer.lr, 0.001)
 
    def on_epoch_begin(self, epoch, logs={}):
        lr_present=K.get_value(self.model.optimizer.lr)
        #print(epoch)
        if (epoch%10==0) and epoch:
        	
            K.set_value(self.model.optimizer.lr, lr_present/((epoch)**0.5))
            print(K.get_value(self.model.optimizer.lr))
            print(lr_present/((epoch)**0.5))

callbacks = [
    Custom_lr()
    ]

In [ ]:
top_model.fit(x_train_bottleneck, y_train,
          epochs=35,
          batch_size=32,
          validation_data=(x_cv_bottleneck, y_cv), callbacks=callbacks,)

Epoch 1/35
4556/4556 [==============================] - 27s 3ms/step - loss: 0.7343 - accuracy: 0.6873 - val_loss: 0.4221 - val_accuracy: 0.8064
Epoch 2/35
4556/4556 [==============================] - 15s 3ms/step - loss: 0.4632 - accuracy: 0.7653 - val_loss: 0.3713 - val_accuracy: 0.8298
Epoch 3/35
4556/4556 [==============================] - 15s 3ms/step - loss: 0.4217 - accuracy: 0.7892 - val_loss: 0.3393 - val_accuracy: 0.8499
Epoch 4/35
4556/4556 [==============================] - 15s 3ms/step - loss: 0.3929 - accuracy: 0.8025 - val_loss: 0.3270 - val_accuracy: 0.8564
Epoch 5/35
4556/4556 [==============================] - 14s 3ms/step - loss: 0.3722 - accuracy: 0.8140 - val_loss: 0.3084 - val_accuracy: 0.8661
Epoch 6/35
4556/4556 [==============================] - 15s 3ms/step - loss: 0.3543 - accuracy: 0.8240 - val_loss: 0.2927 - val_accuracy: 0.8748
Epoch 7/35
4556/4556 [==============================] - 15s 3ms/step - loss: 0.3403 - accuracy: 0.8285 - val_loss: 0.2855 - val_ac

In [ ]:
top_model.save(savedModelPath + 'top_model_full_data_custom_lr_weights.h5') # best


Fine Tune Model

In [7]:
# load VGG16
vgg_model=vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3))

58900480/58889256 [==============================] - 1s 0us/step


In [8]:
model_aug=Sequential()
model_aug.add(vgg_model)

In [9]:
top_model=Sequential()
top_model.add(Flatten(input_shape=(2, 2, 512)))
top_model.add(Dense(64, activation='relu'))
top_model.add(Dense(1, activation='sigmoid'))
top_model.load_weights(savedModelPath + 'top_model_full_data_custom_lr_weights.h5')

model_aug.add(top_model)

In [10]:
for layer in model_aug.layers[0].layers[:17]:
    layer.trainable=False

In [11]:
model_aug.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=1e-6), metrics=['accuracy'])

In [12]:
model_aug.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 131201    
Total params: 14,845,889
Trainable params: 2,491,009
Non-trainable params: 12,354,880
_________________________________________________________________


In [13]:
model_aug.fit(x_train, y_train, epochs=1, batch_size=1, validation_data=(x_cv, y_cv), verbose=1)

145787/145787 [==============================] - 15050s 103ms/step - loss: 0.1053 - accuracy: 0.9541 - val_loss: 0.2323 - val_accuracy: 0.9282


In [14]:
model_aug.save(savedModelPath + 'fine_tuned_model_adam_weights_new.h5')